# Convert the [FIRE](https://galaxies.northwestern.edu/fire-simulations/) data from the isolated galaxy to use in Uniview

I will use the isolated galaxy model that we've used for testing [Firefly](https://github.com/ageller/Firefly) here.  That simulation’s “proper name” is control_G4_020 and was run by Paul Torrey as part of his nuclear star formation paper.  Note, the bi-conical outflow in the simulation  is due to AGN feedback, so scientifically this isn’t accurate.  But the look should be fine for our purposes. 

In [23]:
import csv
import pandas as pd
import numpy as np

# firefly files
import sys
sys.path.insert(0, '/Users/ageller/Visualizations/Firefly/static/data')

from firefly_api.FIREreader import FIREreader


### Load in the data

In [15]:
reader = FIREreader(
    #snapdir = "/Users/agurvich/research/snaps/Control_G4_20/snapdir_050/",
    snapdir = "/Users/ageller/Visualizations/FIREdata/snapdir_050/",
    snapnum = 50,
    ptypes=['PartType0','PartType4'],
    UInames=['Gas','Stars'],
    dec_factors=[1,1],
    returnKeys=['Density','Velocities','Temperature'],
    doMags=[0,0,0],
    doLogs=[1,0,1],
    filterFlags=[1,1,1], ## Note: previously Velocities were automatically included, now you must mark 1 to filter 
    colormapFlags=[1,1,1], 
    JSONdir='isolatedGalaxy_s50',
    doValidate = False)
        
reader.loadData()

Loading ptype PartType0
/Users/ageller/Visualizations/FIREdata/snapdir_050/snapshot_050.0.hdf5
Loading ptype PartType4
/Users/ageller/Visualizations/FIREdata/snapdir_050/snapshot_050.0.hdf5


[Particle Group of Gas
 Contains 1066668 (1066668 after dec) particles and 3 arrays,
 Particle Group of Stars
 Contains 514118 (514118 after dec) particles and 1 arrays]

In [16]:
outDict = {"parts":{}, "options":None}
for pg in reader.particleGroups:
    outDict['parts'][pg.UIname] = pg.outputToDict()

In [17]:
outDict

{'parts': {'Gas': {'Coordinates': array([[ -8.29072  ,   6.6803894, -12.610132 ],
          [ -7.0236144,   7.1231937, -10.539261 ],
          [ -5.641641 ,   7.237127 , -11.150634 ],
          ...,
          [-40.84718  ,  -6.8125515,  61.42491  ],
          [-45.157013 ,  -9.304246 ,  48.431717 ],
          [-46.971127 , -13.404824 ,  88.89448  ]], dtype=float32),
   'log10Density': array([ -6.9275417,  -6.844107 ,  -6.8276525, ...,  -9.61074  ,
           -9.32994  , -10.429991 ], dtype=float32),
   'Velocities': array([[-392.07446  ,  435.0771   , -334.3628   ],
          [-295.4825   ,  381.93094  , -392.7964   ],
          [ -67.50525  ,  269.895    , -299.97534  ],
          ...,
          [ -82.58224  ,  -55.27619  ,   56.62399  ],
          [ -71.25266  ,  -50.233826 ,    7.6796403],
          [   5.660646 ,  -17.029835 ,  -11.176822 ]], dtype=float32),
   'log10Temperature': array([6.746951 , 6.773773 , 6.7626405, ..., 5.1724434, 5.143463 ,
          4.2304893], dtype=float32

### Write to output files

* We want to take all the high velocity stuff but only some fraction of the rest

In [26]:
#gas
csvfile = open("../data/FIREgas.raw", 'wt')
csvwriter = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)

coords = outDict['parts']['Gas']['Coordinates']
velocities = outDict['parts']['Gas']['Velocities']
temperatures = outDict['parts']['Gas']['log10Temperature']
densities = outDict['parts']['Gas']['log10Density']

vlim = 500
dFac = 0.1
for c,v,t,d in zip(coords, velocities, temperatures, densities):
    doPrint = False
    vmag = (v[0]**2. +  v[1]**2. + v[2]**2.)**0.5
    if (vmag > 500):
        doPrint = True
    else:
        xx = np.random.random()
        if (xx < dFac):
            doPrint = True
    if (doPrint):
        csvwriter.writerow([c[0], c[1], c[2], 
                            v[0], v[1], v[2],
                            vmag, t, d])
csvfile.close()

In [27]:
#stars
csvfile = open("../data/FIREstars.raw", 'wt')
csvwriter = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)

coords = outDict['parts']['Stars']['Coordinates']
velocities = outDict['parts']['Gas']['Velocities']

dFac = 0.1
for c,v in zip(coords, velocities):
    doPrint = False
    xx = np.random.random()
    if (xx < dFac):
        doPrint = True
        
    if (doPrint):
        csvwriter.writerow([c[0], c[1], c[2], 
                            v[0], v[1], v[2],
                            vmag, 1, 2])
csvfile.close()

## Look at this in Firefly

In [33]:
from simpleReader import simpleReader
import requests
from IPython.display import IFrame

## Start the server

In [41]:
%%bash --bg --proc bg_proc
python /Users/ageller/Visualizations/Firefly/FireflyFlaskApp.py

## Show Firefly in an IFrame

In [42]:
url = "http://localhost:5000/combined"
IFrame(url, width=1000, height=700)

### Format the data and send it to Firefly

In [44]:
df = pd.read_csv('../data/FIREgas.raw', delimiter=' ', 
#df = pd.read_csv('../data/FIREstars.raw', delimiter=' ', 
                 names=['c0', 'c1','c2', 'v0', 'v1', 'v2', 'vmag', 't', 'd'], header=None)
name = "gas"
coords = np.vstack((df['c0'].values, df['c1'].values,df['c2'].values)).T
                  
outDict = simpleReader(name, coords, [1,0,0,0.7], 0.5)
print(outDict['parts'][name]['Coordinates'].shape)

s = pd.Series(outDict).to_json()
requests.post("http://localhost:5000/data_input", json=s)

(111768, 3)


<Response [200]>

*To kill the Firefly server.*

In [45]:
# uncomment the next line
! ps aux | grep Firefly | awk '{print $2}' | xargs kill

kill: 10694: No such process
